## Preamble

In [1]:
import sys

# Allows imports from the scripts directory.
sys.path.append('../scripts')

## Prepare the data

In [2]:
label2id = {
    'ISSUE': 0,
    'NONE': 1,
    'STA': 2,
    'ANALYSIS': 3,
    'PRE_RELIED': 4,
    'RATIO': 5,
    'RPC': 6,
    'PRE_NOT_RELIED': 7,
    'ARG_PETITIONER': 8,
    'PREAMBLE': 9,
    'RLC': 10,
    'ARG_RESPONDENT': 11,
    'FAC': 12
}

id2label = {v: k for k, v in label2id.items()}

In [3]:
from datautils import RRDataModule, RRBatcher

DPATH = "../data/train.json"
MPATH = "sentence-transformers/all-MiniLM-L6-v2"

batcher = RRBatcher(MPATH)
datamodule = RRDataModule(DPATH, batcher, label2id)
datamodule.setup()


c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare the model

In [4]:
from pathlib import Path
from modelling import CoherenceAwareSentenceEmbedder
from pytorch_lightning.callbacks import(
    EarlyStopping,
    ModelCheckpoint
)

SAVE_PATH = "../models/coherence-aware/all-MiniLM-L6-v2"
CKPT = Path(SAVE_PATH).joinpath("masked-epoch=11-val_sem_loss=0.29.ckpt")


surrogate_imp=0.2
try:
    model = CoherenceAwareSentenceEmbedder.load_from_checkpoint(CKPT, num_classes=len(label2id))
    print("Loaded checkpoint")
except:
    model = CoherenceAwareSentenceEmbedder(MPATH,  num_classes=len(label2id), surrogate_imp=surrogate_imp, triplet_margin=1.0)
    print("Loaded fresh model")

checkpoint_callback = ModelCheckpoint(
    SAVE_PATH,
    filename=f'imp={surrogate_imp}-{{epoch}}-{{val_sem_loss:.2f}}',
    monitor='val_sem_loss',
    # save_weights_only=True,
)
early_stop_callback = EarlyStopping(
    monitor="val_sem_loss",
    min_delta=8e-4, patience=3,
    verbose=False,
    mode="min"
)

AttributeError: 'CoherenceAwareSentenceEmbedder' object has no attribute 'backbone'

## Train the model

In [ ]:
from pytorch_lightning import Trainer, seed_everything


seed_everything(42, workers=True)

trainer = Trainer(
    max_epochs=-1,
    deterministic=True,
    accumulate_grad_batches=4,
    callbacks=[checkpoint_callback, early_stop_callback],
    accelerator='gpu',
    log_every_n_steps=16
)

trainer.fit(model, datamodule)